# EfficientNet-Lite4 - Transferred Learning

---
Link to TensorFlow Hub
https://tfhub.dev/tensorflow/efficientnet/lite4/classification/2

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
import pathlib


2023-05-17 21:15:27.497194: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-17 21:15:27.497221: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
num_classes = 6
# Load the EfficientNet-Lite model from TensorFlow Hub
module_url = "https://tfhub.dev/tensorflow/efficientnet/lite4/classification/2"
model = Sequential([
    hub.KerasLayer(module_url, trainable=False, input_shape=(224, 224, 3)),
    Dense(num_classes, activation='softmax')
])
print("done")

done


In [7]:
train_dir = '../SFSU_Art/training'
validation_dir = '../SFSU_Art/validation'
train_dir = pathlib.Path(train_dir)
validation_dir = pathlib.Path(validation_dir)

print(train_dir)
print(validation_dir)

../SFSU_Art/training
../SFSU_Art/validation


In [8]:
batch_size = 10
img_height = 224
img_width = 224

# Training data  
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

#Validation Data
val_ds = tf.keras.utils.image_dataset_from_directory(
  validation_dir, 
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

train_classnames = train_ds.class_names
#with open('./models/label.txt', 'w') as f:
#    f.writelines(str(item) + '\n' for item in class_names)



Found 206 files belonging to 7 classes.
Using 165 files for training.
Found 70 files belonging to 7 classes.
Using 14 files for validation.


# Fixing the number output shape

---


In [9]:
# Get the output layer of the model
output_layer = model.layers[-1]

# Create a new input layer with 1 unit
new_input_layer = Dense(1)(output_layer.output)

# Create a new output layer with the desired number of units and activation function
new_output_layer = Dense(6, activation='softmax')(new_input_layer)

# Create a new model with the same input and all layers except the original output layer
new_model = keras.Model(inputs=model.input, outputs=new_output_layer, name='new_model')


In [10]:
new_model = build_model(7)

NameError: name 'build_model' is not defined

In [11]:
# Compile the model with a categorical crossentropy loss function and an Adam optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1000)              13118936  
                                                                 
 dense_1 (Dense)             (None, 6)                 6006      
                                                                 
Total params: 13,124,942
Trainable params: 6,006
Non-trainable params: 13,118,936
_________________________________________________________________


In [33]:
# print the configuration of the previous layer
previous_layer_config = model.layers[-2].get_config()
print(previous_layer_config)


{'name': 'keras_layer', 'trainable': False, 'batch_input_shape': (None, 260, 260, 3), 'dtype': 'float32', 'handle': 'https://tfhub.dev/tensorflow/efficientnet/lite4/classification/2', 'signature': 'default', 'output_key': 'default'}


In [37]:
from tensorflow.keras.layers import Reshape

# Get the output layer of the model
output_layer = model.layers[-1]

# Create a new input layer with the same shape as the output of the original model
new_input_layer = Dense(1)(output_layer.output)

# Reshape the new input layer to match the desired output shape
new_input_layer_reshaped = Reshape((1,))(new_input_layer)

# Create a new output layer with the desired number of units and activation function
new_output_layer = Dense(6, activation='softmax')(new_input_layer_reshaped)

# Create a new model with the same input and all layers except the original output layer
new_model = keras.Model(inputs=model.input, outputs=new_output_layer, name='new_model')


In [12]:
# Train the model on a new dataset
epochs = 1
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

ValueError: in user code:

    File "/home/emanuelf/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/home/emanuelf/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/emanuelf/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/home/emanuelf/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/emanuelf/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "/home/emanuelf/anaconda3/lib/python3.9/site-packages/keras/engine/compile_utils.py", line 201, in __call__
        lo